<a href="https://colab.research.google.com/github/MorningStarTM/brain_cancer_detection/blob/main/brain_tumor_classification_with_simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#importing libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.utils import shuffle
from glob import glob
from tensorflow.keras.layers import*
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import*
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [117]:
#parameters
H, W = 224, 224
channel = 3
num_class = 3
batch_size = 16
class_names = ["glioma", "meningioma", "notumor", "pituitary"]

#Data pipeline

In [190]:
#load the dataset
def load_data(path, split=0.7):
  images = tf.data.Dataset.list_files(glob(os.path.join(path, "*", "*.jpg")))
  #split size
  split_size = int(len(images) * split)

  #train size
  x_train = images.take(split_size)
  #sample size
  sample = images.skip(split_size)
  sample_split = int(len(sample)*0.5)

  #test size
  x_test = sample.take(sample_split)
  #valid size
  x_valid = sample.skip(sample_split)

  return x_train, x_valid, x_test

In [189]:
#extract the name of image from path
def get_label(path):
  #spliting string
  name = tf.strings.split(path, os.path.sep)[-2]
  return name

In [228]:
def read_image(path):
  #get label
  label = get_label(path)
  #read file
  img = tf.io.read_file(path)
  #read image
  img = tf.image.decode_jpeg(img)
  #resize the image
  img = tf.image.resize(img, [W, H])
  #scale image
  #img = img / 255.0
  
  return img, label

In [241]:
def tf_dataset(dataset, batch=8):
  dataset = dataset.map(read_image)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(8)
  return dataset

In [242]:
path = "/content/drive/MyDrive/DataSet/brain_cancer/Training"

In [243]:
x_train, x_valid, x_test = load_data(path)
print(f"Train:{len(x_train)} - Test:{len(x_test)} - Valid:{len(x_valid)}")

Train:4012 - Test:860 - Valid:860


In [244]:
df = tf_dataset(x_train)

In [245]:
for img, lab in df.take(1):
  print(img.shape)

InvalidArgumentError: ignored